# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# очень часто сайты могут ограничивать частые запросы к себе,поэтому нужно задерживать исполнение
import time

In [38]:
KEYWORDS = ['erp-системы', 'удалённая работа', 'информационная безопасность']

In [33]:
URL = 'https://habr.com/ru/all'
req = requests.get(URL)
#req.text

In [34]:
soup = BeautifulSoup(req.text, 'html.parser')
#soup

In [35]:
# извлекаем посты
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/Seleditor/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/f21/4b9/a3d/f214b9a3df13c490cbb6110700b95965.png" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">Seleditor</span>
 </a>
 <span class="post__time">сегодня в 13:19</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/selectel/blog/513434/">Прототип Starship успешно прошел огневые испытания</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/selectel/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Блог компании Selectel'); }" rel="nofollow" title="Вы н

In [39]:
list_ = []

for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link') #выделение тегов к статьям
    
    for hub in hubs:
            hub_lower = hub.text.lower()
            # ищем вхождение хотя бы одного желаемого хаба
            if any([hub_lower in words for words in KEYWORDS]):
                # пост нам интересен - делаем с ним все что захотим: можно отправит в телеграм уведомление, можно на почту и т.п.
                title_element = post.find('a', class_='post__title_link')
                a = title_element.attrs.get('href')
                soup = BeautifulSoup(requests.get(a).text, 'html.parser')
                time.sleep(0.3)
                text = soup.find('div', class_='post__text post__text-html post__text_v1').text
                title_element2 = post.find('a', class_='post__title_link').text
                date = post.find('span', class_='post__time').text
                row = {'date': date, 'title': title_element2, 'ссылка': a, 'text': text}
                list_.append(row)
                # так как пост уже нам подошел - дальше нет смысла проверять хабы
                break

In [40]:
df = pd.DataFrame(list_)
df

,date,title,ссылка,text
0,вчера в 18:03,"HackTheBox. Прохождение Oouch. OAuth2, RCE в u...",https://habr.com/ru/post/513378/,\r\nПродолжаю публикацию решений отправленных ...
1,вчера в 15:43,Небольшое расследование расследования по делу ...,https://habr.com/ru/post/513402/,"Наверное, все помнят, как около 2 недель назад..."
2,вчера в 14:20,Автоматизация бюджетирования: в чем проблема?,https://habr.com/ru/post/513396/,"\n\r\nЭто вводная статья о том, что такое «авт..."
3,вчера в 13:54,"Облачный TL;DR: что дает open source, почему р...",https://habr.com/ru/company/1cloud/blog/513354/,Вместо привычных дайджестов избранных постов и...


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [3]:
URL2 = 'https://digibody.avast.com/v1/web/leaks'
params = {'email': "xxx@x.ru"}
#URL2 = 'https://www.avast.com'
#URL2 = 'https://digibody.avast.com/v1/web/leaks'

In [4]:
res = requests.post(URL2, json=params)
res

<Response [200]>

In [5]:
res.text

'{"status":"ok","value":[{"leak_id":"037a277e-d484-4c34-8473-904024e489e1","username":"xxx@x.ru","domain":"","passwords":[{"password":null,"encrypted":false}],"leak_info":{"id":"037a277e-d484-4c34-8473-904024e489e1","date":1513900800000,"title":"Combolist of 1.4 Billion Credentials","description":"The proliferation of stolen or leaked databases has given rise to credential stuffing, a fairly simple technique in which criminals load lists of stolen credentials, called combo lists, into automated brute-forcing tools to test credentials en masse. These tools test stolen passwords against thousands of targeted websites and applications until there is a match.\\n\\nThis particular combo list was likely compiled over time from a variety of public and private breaches. It contains approximately 1.4 billion email and password records. Criminals are actively leveraging this list, along with credential stuffing tools, to gain unauthorized access to targeted websites. This list has been made publ

In [6]:
import json

In [25]:
vocab = json.loads(res.text)
vocab

{'status': 'ok',
 'value': [{'leak_id': '037a277e-d484-4c34-8473-904024e489e1',
   'username': 'xxx@x.ru',
   'domain': '',
   'passwords': [{'password': None, 'encrypted': False}],
   'leak_info': {'id': '037a277e-d484-4c34-8473-904024e489e1',
    'date': 1513900800000,
    'title': 'Combolist of 1.4 Billion Credentials',
    'description': 'The proliferation of stolen or leaked databases has given rise to credential stuffing, a fairly simple technique in which criminals load lists of stolen credentials, called combo lists, into automated brute-forcing tools to test credentials en masse. These tools test stolen passwords against thousands of targeted websites and applications until there is a match.\n\nThis particular combo list was likely compiled over time from a variety of public and private breaches. It contains approximately 1.4 billion email and password records. Criminals are actively leveraging this list, along with credential stuffing tools, to gain unauthorized access to tar

In [8]:
type(vocab)
vocab2 = vocab['value']
vocab2

[{'leak_id': '037a277e-d484-4c34-8473-904024e489e1',
  'username': 'xxx@x.ru',
  'domain': '',
  'passwords': [{'password': None, 'encrypted': False}],
  'leak_info': {'id': '037a277e-d484-4c34-8473-904024e489e1',
   'date': 1513900800000,
   'title': 'Combolist of 1.4 Billion Credentials',
   'description': 'The proliferation of stolen or leaked databases has given rise to credential stuffing, a fairly simple technique in which criminals load lists of stolen credentials, called combo lists, into automated brute-forcing tools to test credentials en masse. These tools test stolen passwords against thousands of targeted websites and applications until there is a match.\n\nThis particular combo list was likely compiled over time from a variety of public and private breaches. It contains approximately 1.4 billion email and password records. Criminals are actively leveraging this list, along with credential stuffing tools, to gain unauthorized access to targeted websites. This list has been

In [98]:
type(vocab2)

list

In [18]:
df3 = pd.DataFrame(vocab2)
df3[['username', 'leak_date', 'leak_info']]

,username,leak_date,leak_info
0,xxx@x.ru,1513900800000,"{'id': '037a277e-d484-4c34-8473-904024e489e1',..."
1,xxx@x.ru,1549411200000,"{'id': 'cf7afbbb-7695-4f97-b755-3d63df87828f',..."
2,aleksey1688,1485820800000,"{'id': 'ee4ffcf9-e69f-49b7-9c61-70f94fa82ad3',..."
3,xixixixixixixixixixi,1487030400000,"{'id': '053fe29c-422d-4b69-bce2-1ce4904af697',..."
4,xxx@x.ru,1488931200000,"{'id': '7f6e3b64-9951-458f-b565-8ff02baf1a59',..."
5,xxx@x.ru,1549411200000,"{'id': '71db3f29-006d-4158-a3f0-8102652fbf57',..."
6,xxx@x.ru,1549411200000,"{'id': '83571809-585b-4fff-a72f-cdf0bc345916',..."
7,stalkera116,1487030400000,"{'id': '2702007e-b243-4836-b1d6-a2d85f03b469',..."
8,xxx@x.ru,1477008000000,"{'id': '7f667060-f7c1-4f48-b93d-ecb4d066ba4b',..."
9,xxx@x.ru,1548720000000,"{'id': 'b15edc13-85b2-4c53-a54a-fee2d6b861ea',..."


In [29]:
z = {}
for el in vocab2:
    el
z = el
z
    

{'leak_id': '8c9faf25-41e0-431c-8090-93950692cc61',
 'username': 'xcbear',
 'domain': 'imesh.com',
 'passwords': [{'password': None, 'encrypted': True}],
 'leak_info': {'id': '8c9faf25-41e0-431c-8090-93950692cc61',
  'date': 1477180800000,
  'title': 'iMesh',
  'description': 'In June 2016, a cache of over 51 million user credentials from the online sharing service iMesh appeared for sale on a dark web marketplace.  The database contained user email addresses, usernames, passwords, IP addresses and location.',
  'number_of_entries': 50913222,
  'source_references': [],
  'media_references': ['https://www.zdnet.com/article/51-million-imesh-file-sharing-accounts-for-sale-dark-web/'],
  'domains': ['imesh.com'],
  'picture_url': None,
  'service_name': None},
 'marked_resolved': False,
 'leak_date': 1477180800000,
 'passwords_count': 1}

In [30]:
df4 = pd.DataFrame(z)
df4

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
